In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from ivan.data.dataset import ElectroInterDataset
from ivan.model.unet import UNet

import numpy as np
import torch

In [3]:
DATA_PATH = "../data/processed"
EPOCHS = 1000
DEVICE = "cuda"

In [4]:
dataset = ElectroInterDataset(DATA_PATH)

In [6]:
model = UNet(4, 17).to(DEVICE)
opt = torch.optim.SGD(model.parameters(), lr=1, momentum=0.9)

In [13]:
for epoch in EPOCHS:
    for i in range(len(dataset)):
        image, label = dataset[i]
        # model.zero_grad()
        # predict = model(image)
        # loss = loss_func(predict, label)
        # loss.bacward()
        # opt.step()
        print(i)

0
1
